## Preprocessing

In this first phase we are focusing on getting only the High Resolutions images for the patients that respect the neccessary quality.

In [2]:
import os
import glob
import json
import pandas as pd
from collections import defaultdict
import shutil

Getting only high resolution MRI by checking:
- SliceThickness
- ImageOrientationPatientDICOM

In [15]:
def get_sagittal_slices(input_dir, output_dir):
    json_files = glob.glob(f'{input_dir}/**/*.json', recursive=True)

    des = output_dir
    os.makedirs(des, exist_ok=True)
    
    for json_file in json_files:
        try:
            with open(json_file) as f:
                data = json.load(f)

            if 'SliceThickness' in data and data['SliceThickness'] <= 1.5:
                if 'ImageOrientationPatientDICOM' in data and data['ImageOrientationPatientDICOM'][1] > 0.9 and abs(data['ImageOrientationPatientDICOM'][5]) > 0.9:
                    
                    nii_file = json_file.replace('.json', '.nii.gz')
                    
                    os.system(f"cp '{json_file}' {des}")
                    os.system(f"cp '{nii_file}' {des}")

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in {json_file}: {e}")
        except Exception as e:
            print(f"Error processing {json_file}: {e}")

In [16]:
input_dir = '/scratch/Costanza/ADNI_DDPM_NIfTI'
output_dir = '/scratch/Costanza/ADNI_DDPM_Sagittal'
get_sagittal_slices(input_dir, output_dir)

Extration of the patient ID

In [3]:
def extract_patient_id(filename):
    base_name = os.path.basename(filename)
    patient_id = base_name.split('_')[2]
    return patient_id

Get only the paired patients by checking:

- Total Number of patients
- Patients with both Magnetic Field Strength (1.5T and 3T) for same Series Description

In [9]:
def find_all_patients(input_dir):
    json_files = glob.glob(f'{input_dir}/**/*.json', recursive=True)
    all_patients = {}

    for json_file in json_files:
        try:
            with open(json_file) as f:
                data = json.load(f)
                
            patient_id = extract_patient_id(json_file)
            series_description = data.get('SeriesDescription', 'Unknown')

            # Initialize the patient's entry if not already in the dictionary
            if patient_id not in all_patients:
                all_patients[patient_id] = {}

            if series_description not in all_patients[patient_id]:
                all_patients[patient_id][series_description] = {'3T': False, '1.5T': False}

            if 'MagneticFieldStrength' in data:
                if data['MagneticFieldStrength'] == 3.0:
                    all_patients[patient_id][series_description]['3T'] = True
                elif data['MagneticFieldStrength'] == 1.5:
                    all_patients[patient_id][series_description]['1.5T'] = True

        except Exception as e:
            print(f"Error processing {json_file}: {e}")

    return all_patients

input_dir = '/scratch/Costanza/ADNI_DDPM_Sagittal'
all_patients = find_all_patients(input_dir)

# Count patients who have both 1.5T and 3T scans for the same SeriesDescription
patients_with_both = 0
patients_with_3T_only = 0
patients_with_1_5T_only = 0

for patient_id, series_dict in all_patients.items():
    has_both = any(info['3T'] and info['1.5T'] for info in series_dict.values())
    has_3T_only = any(info['3T'] and not info['1.5T'] for info in series_dict.values())
    has_1_5T_only = any(info['1.5T'] and not info['3T'] for info in series_dict.values())
    
    if has_both:
        patients_with_both += 1
    elif has_3T_only:
        patients_with_3T_only += 1
    elif has_1_5T_only:
        patients_with_1_5T_only += 1

total_patients = len(all_patients)
patients_without_both = total_patients - (patients_with_both + patients_with_3T_only + patients_with_1_5T_only)

print(f'The total number of patients in this dataset directory {input_dir} is {total_patients}')
print(f'Number of patients with both 3T and 1.5T scans for the same SeriesDescription: {patients_with_both}')
print(f'Number of patients with only 3T scans: {patients_with_3T_only}')
print(f'Number of patients with only 1.5T scans: {patients_with_1_5T_only}')
print(f'Number of patients without both 3T and 1.5T scans for the same SeriesDescription: {patients_without_both}')

The total number of patients in this dataset directory /scratch/Costanza/ADNI_DDPM_Sagittal is 99
Number of patients with both 3T and 1.5T scans for the same SeriesDescription: 44
Number of patients with only 3T scans: 53
Number of patients with only 1.5T scans: 2
Number of patients without both 3T and 1.5T scans for the same SeriesDescription: 0


Table of SeriesDescriptions available for patient that have both 1.5T and 3T MagneticFieldStrength 

In [133]:
file_path = '/scratch/Costanza/Check/MFS&SD_sagittal_check.csv'
df = pd.read_csv(file_path)

filtered_df = df.groupby(['PatientID', 'SeriesDescription'])['MagneticFieldStrength'].apply(lambda x: set(x) == {1.5, 3.0}).reset_index()

filtered_df = filtered_df[filtered_df['MagneticFieldStrength']]

count_series_description = filtered_df['SeriesDescription'].value_counts().reset_index()
count_series_description.columns = ['SeriesDescription', 'PatientCount']

total_patient = count_series_description['PatientCount'].sum()

count_series_description


,SeriesDescription,PatientCount
0,MPRAGE,33
1,MPRAGE Repeat,13
2,MP-RAGE REPEAT,11
3,MP-RAGE,10
4,MPRAGE REPEAT,1


The total of the column Patient Count is 68, but there are only 44 unique patients in the CSV file, it indicates that some patients have multiple series descriptions associated with them.

Function that clean the dataset and get just the patients that we need. From 99 we found just 44 patients with the characteristics that we want.

In [5]:
def get_patients(input_dir, output_dir):
    json_files = glob.glob(f'{input_dir}/**/*.json', recursive=True)
    
    patient_scans = defaultdict(lambda: defaultdict(list))
    
    for json_file in json_files:
        try:
            with open(json_file) as f:
                data = json.load(f)

            if 'MagneticFieldStrength' in data and 'SeriesDescription' in data:
                patient_id = extract_patient_id(json_file)
                field_strength = data['MagneticFieldStrength']
                series_description = data['SeriesDescription']
                patient_scans[patient_id][field_strength].append((series_description, json_file))

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in {json_file}: {e}")
        except Exception as e:
            print(f"Error processing {json_file}: {e}")
    
    patients_with_both = {
        patient: field_strengths 
        for patient, field_strengths in patient_scans.items() 
        if 1.5 in field_strengths and 3.0 in field_strengths
    }

    filtered_patients = {}
    for patient, field_strengths in patients_with_both.items():
        series_descriptions_1_5 = set(desc for desc, _ in field_strengths[1.5])
        series_descriptions_3_0 = set(desc for desc, _ in field_strengths[3.0])
        common_series_descriptions = series_descriptions_1_5.intersection(series_descriptions_3_0)
        
        if common_series_descriptions:
            filtered_patients[patient] = field_strengths
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for patient, field_strengths in filtered_patients.items():
        series_descriptions_1_5 = {desc: file for desc, file in field_strengths[1.5]}
        series_descriptions_3_0 = {desc: file for desc, file in field_strengths[3.0]}
        common_series_descriptions = set(series_descriptions_1_5.keys()).intersection(series_descriptions_3_0.keys())
        
        for series_description in common_series_descriptions:
            json_file_1_5 = series_descriptions_1_5[series_description]
            json_file_3_0 = series_descriptions_3_0[series_description]
            
            image_file_1_5 = json_file_1_5.replace('.json', '.nii.gz') 
            image_file_3_0 = json_file_3_0.replace('.json', '.nii.gz')
            
            patient_output_dir = os.path.join(output_dir, patient)
            if not os.path.exists(patient_output_dir):
                os.makedirs(patient_output_dir)
            
            json_file_1_5_new = os.path.join(patient_output_dir, f'{os.path.basename(json_file_1_5).replace(".json", "")}_1.5T.json')
            image_file_1_5_new = os.path.join(patient_output_dir, f'{os.path.basename(image_file_1_5).replace(".nii.gz", "")}_1.5T.nii.gz')
            json_file_3_0_new = os.path.join(patient_output_dir, f'{os.path.basename(json_file_3_0).replace(".json", "")}_3.0T.json')
            image_file_3_0_new = os.path.join(patient_output_dir, f'{os.path.basename(image_file_3_0).replace(".nii.gz", "")}_3.0T.nii.gz')
            
            shutil.copy(json_file_1_5, json_file_1_5_new)
            shutil.copy(image_file_1_5, image_file_1_5_new)
            shutil.copy(json_file_3_0, json_file_3_0_new)
            shutil.copy(image_file_3_0, image_file_3_0_new)
    
    num_patients = len(filtered_patients)
    return num_patients

input_dir = '/scratch/Costanza/ADNI_DDPM_Sagittal'
output_dir = '/scratch/Costanza/ADNI_DDPM_S_Clean'
num_patients = get_patients(input_dir, output_dir)
print(f"Number of unique patients with both 1.5T and 3T scans with common series descriptions: {num_patients}")

Number of unique patients with both 1.5T and 3T scans with common series descriptions: 44


In [13]:
def check_patient_folder(patient_folder):
    nii_files = glob.glob(os.path.join(patient_folder, '*.nii.gz'))

    image_3T_files = []
    image_1_5T_files = []

    for nii_file in nii_files:
        if '3.0T' in nii_file:
            image_3T_files.append(nii_file)
        elif '1.5T' in nii_file:
            image_1_5T_files.append(nii_file)

    num_pairs = min(len(image_3T_files), len(image_1_5T_files))

    if not image_3T_files:
        print(f"No 3.0T images found in {patient_folder}")
    if not image_1_5T_files:
        print(f"No 1.5T images found in {patient_folder}")

    return num_pairs, image_1_5T_files, image_3T_files

def check_all_patient_folders(input_dir):
    patient_folders = [f.path for f in os.scandir(input_dir) if f.is_dir()]
    complete_pairs_count = 0
    multiple_pairs_count = 0
    for patient_folder in patient_folders:
        num_pairs, _, _ = check_patient_folder(patient_folder)
        if num_pairs > 0:
            complete_pairs_count += 1
            if num_pairs > 1:
                multiple_pairs_count += 1
                print(f"Patient {os.path.basename(patient_folder)} has {num_pairs} pairs")
        else:
            print(f"Incomplete pairs in {patient_folder}")

    print(f"Total patients with complete pairs: {complete_pairs_count}")
    print(f"Total patients with multiple pairs: {multiple_pairs_count}")
    print(f"Total patients checked: {len(patient_folders)}")
    
input_dir = '/scratch/Costanza/ADNI_DDPM_S_Clean'
check_all_patient_folders(input_dir)


Patient 1169 has 2 pairs
Patient 1276 has 2 pairs
Patient 1241 has 2 pairs
Patient 0622 has 2 pairs
Patient 0058 has 2 pairs
Patient 0602 has 2 pairs
Patient 1251 has 2 pairs
Patient 1267 has 2 pairs
Patient 1206 has 2 pairs
Patient 0031 has 2 pairs
Patient 1288 has 2 pairs
Patient 0926 has 2 pairs
Patient 1190 has 2 pairs
Patient 1256 has 2 pairs
Patient 0605 has 2 pairs
Patient 0677 has 2 pairs
Patient 0061 has 2 pairs
Patient 0260 has 2 pairs
Patient 0479 has 2 pairs
Patient 0963 has 2 pairs
Patient 1035 has 2 pairs
Patient 1123 has 2 pairs
Patient 0553 has 2 pairs
Patient 1250 has 2 pairs
Total patients with complete pairs: 44
Total patients with multiple pairs: 24
Total patients checked: 44


As last step we are going to clean the dataset from unnecessary pairs of 1.5T and 3.0, remaning just with one couple of pair for patient.

In [11]:
def create_single_pairs_folder(input_dir, output_dir):
    patient_folders = [f.path for f in os.scandir(input_dir) if f.is_dir()]
    
    for patient_folder in patient_folders:
        num_pairs, image_1_5T_files, image_3T_files = check_patient_folder(patient_folder)

        if num_pairs > 0:
            # Only keep the first pair
            selected_1_5T_file = image_1_5T_files[0]
            selected_3T_file = image_3T_files[0]

            patient_id = os.path.basename(patient_folder)
            patient_output_dir = os.path.join(output_dir, patient_id)
            os.makedirs(patient_output_dir, exist_ok=True)

            shutil.copy(selected_1_5T_file, os.path.join(patient_output_dir, os.path.basename(selected_1_5T_file)))
            shutil.copy(selected_3T_file, os.path.join(patient_output_dir, os.path.basename(selected_3T_file)))

            json_1_5T_file = selected_1_5T_file.replace('.nii.gz', '.json')
            json_3T_file = selected_3T_file.replace('.nii.gz', '.json')

            if os.path.exists(json_1_5T_file):
                shutil.copy(json_1_5T_file, os.path.join(patient_output_dir, os.path.basename(json_1_5T_file)))
            if os.path.exists(json_3T_file):
                shutil.copy(json_3T_file, os.path.join(patient_output_dir, os.path.basename(json_3T_file)))

input_dir = '/scratch/Costanza/ADNI_DDPM_S_Clean'
output_dir = '/scratch/Costanza/ADNI_DDPM_S_Single_Pairs'
create_single_pairs_folder(input_dir, output_dir)


### HD-BET Tool

To use HD-BET Skull stripping tool we need to restructure the dataset  without folder for each patients

In [5]:
def flatten_dataset_and_rename(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.nii.gz'):
                parts = file.split('_')
                new_parts = parts[2:]
                if 'T' in new_parts[-1]:
                    new_parts[-1] = new_parts[-1].replace('3.0T', '3_0T').replace('1.5T', '1_5T')
                new_filename = '_'.join(new_parts)
                file_path = os.path.join(root, file)
                shutil.copy(file_path, os.path.join(output_dir, new_filename))

input_dir = '/scratch/Costanza/ADNI_DDPM_S_Single_Pairs'
output_dir = '/scratch/Costanza/ADNI_Flat'
flatten_dataset_and_rename(input_dir, output_dir)

after HD-BET we will remove the mask saved by the tool because we do not need it

In [6]:
def remove_mask_files(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('_mask.nii.gz'):
                count += 1
                file_path = os.path.join(root, file)
                print(f"Removing: {file_path}")
                os.remove(file_path)
    return count

directory = '/scratch/Costanza/ADNI_F_SkullStripping'
num_files = remove_mask_files(directory)
print(f"Total number of mask files removed: {num_files}")


Removing: /scratch/Costanza/ADNI_F_SkullStripping/0023_MPRAGE_20101222091037_2_3_0T_mask.nii.gz
Removing: /scratch/Costanza/ADNI_F_SkullStripping/0311_MPRAGE_20110901102522_2_3_0T_mask.nii.gz
Removing: /scratch/Costanza/ADNI_F_SkullStripping/0622_RMRKH062508_ADNI_14M4_TS_3_20080625123200_3_1_5T_mask.nii.gz
Removing: /scratch/Costanza/ADNI_F_SkullStripping/0767_MPRAGE_20110903152917_2_1_5T_mask.nii.gz
Removing: /scratch/Costanza/ADNI_F_SkullStripping/1194_MPRAGE_20110330095021_3_3_0T_mask.nii.gz
Removing: /scratch/Costanza/ADNI_F_SkullStripping/0156_MPRAGE_20160613134720_2_3_0T_mask.nii.gz
Removing: /scratch/Costanza/ADNI_F_SkullStripping/1288_MPRAGE_20071010152428_2_3_0T_mask.nii.gz
Removing: /scratch/Costanza/ADNI_F_SkullStripping/0405_MN060006594_10982145_20060516102237_3_1_5T_mask.nii.gz
Removing: /scratch/Costanza/ADNI_F_SkullStripping/0156_MPRAGE_20070215135553_2_1_5T_mask.nii.gz
Removing: /scratch/Costanza/ADNI_F_SkullStripping/0260_ADNI_3T12M4_TS_2_20060501130654_2_3_0T_mask.nii